## Example of faking classes with a closure

In [1]:
import sys

In [2]:
help(locals)

Help on built-in function locals in module builtins:

locals()
    Return a dictionary containing the current scope's local variables.
    
    NOTE: Whether or not updates to this dictionary will affect name lookups in
    the local scope and vice-versa is *implementation dependent* and not
    covered by any backwards compatibility guarantees.



This code demonstrates a technique that is a way to simulate a class-like behavior using closures in Python. It allows you to create instances that behave like classes but use closures and functions instead of traditional class-based structures.

Let's break down the code and explain each part:

1. **`ClosureInstance` class**:
   This class is used to create instances that mimic class instances using closures. The `__init__` method takes a dictionary of local variables (`locals`) and updates the instance dictionary with callables (functions) from the local variables. It effectively imports functions from the local scope into the instance.

2. **`__len__` method**:
   This method is defined to handle the `__len__()` method in the instance. It redirects the `__len__()` call to the `__len__` function provided in the local scope.

3. **`Stack` function**:
   This function is an example of how to create a "class-like" structure using closures. It defines a stack with `push`, `pop`, and `__len__` functions. It then returns an instance created using `ClosureInstance`, which will have these functions as its methods.


This code allows you to create instances that behave like objects with methods (`push`, `pop`, `__len__`), but it uses closures and functions instead of a traditional class definition. It's a demonstration of how Python's flexibility allows for creative solutions, even when deviating from the standard class-based paradigm.

In [3]:
from typing import Any, List

In [4]:
class ClosureInstance:
    def __init__(self, locals=None) -> None:
        if locals is None:
            locals = sys._getframe(1).f_locals

        # Update instance dictionary with callables
        self.__dict__.update((key,value) for key, value in locals.items()
                             if callable(value))

    # Redirect special methods
    def __len__(self) -> int:
        return self.__dict__['__len__']()

In [5]:
# Example use

def Stack() -> ClosureInstance:
    items: List[Any] = []

    def push(item: Any) -> None:
        items.append(item)

    def pop() -> Any:
        return items.pop()

    def __len__() -> int:
        return len(items)

    return ClosureInstance()


In [6]:
s = Stack()
print(s)

In [7]:
s.push(10)
s.push(20)
s.push('Hello')

In [8]:
len(s)

3

In [9]:
s.pop()

'Hello'

In [10]:
s.pop()

20

In [11]:
s.pop()

10

## Without the `ClosureInstance`
Without using the `ClosureInstance` class, the `Stack` function would directly return a dictionary or a custom object where the methods (`push`, `pop`, `__len__`) would be defined as keys in the dictionary or attributes of the custom object.

In [12]:
def StackFunc() -> dict:
    items: List[Any] = []

    def push(item: Any) -> None:
        items.append(item)

    def pop() -> Any:
        return items.pop()

    def __len__() -> int:
        return len(items)

    return {
        'push': push,
        'pop': pop,
        '__len__': __len__
    }

In [13]:
s = StackFunc()
s

{'push': <function __main__.StackFunc.<locals>.push(item: Any) -> None>,
 'pop': <function __main__.StackFunc.<locals>.pop() -> Any>,
 '__len__': <function __main__.StackFunc.<locals>.__len__() -> int>}

In [14]:
# Need to use dictionary syntax to access functions
s['push'](10)
s['push'](20)
s['push']('Hello')
s['__len__']()

3

In [15]:
s['pop']()

'Hello'

In [16]:
s['pop']()

20

In [17]:
s['__len__']()

1

In this version:
- The `StackFunc` function directly returns a dictionary with keys `'push'`, `'pop'`, and `'__len__'`, where each key corresponds to a function.
- To call the functions, dictionary syntax is used, similar to accessing methods in a class instance.

The main difference is in how the methods are accessed and called. Instead of using method calls like `s.push(10)` and `s.pop()`, you use dictionary syntax like `s['push'](10)` and `s['pop']()`. The idea of encapsulating the functionality within a dictionary remains the same, but the syntax for using the functions changes.

## Example of a normal class

In [18]:
class Stack2:
    def __init__(self) -> None:
        self.items: List[Any] = []

    def push(self, item: Any) -> None:
        self.items.append(item)

    def pop(self) -> Any:
        return self.items.pop()

    def __len__(self) -> int:
        return len(self.items)

In [19]:
from timeit import timeit

In [20]:
s = Stack2()
timeit('s.push(1); s.pop()', 'from __main__ import s')

0.138042292004684

In [21]:
s = Stack()
timeit('s.push(1); s.pop()', 'from __main__ import s')

0.15384129199082963